# Hill-Climbing search
code implementation from [`aima-python`](https://github.com/aimacode/aima-python/blob/master/search.py)

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from search import Problem, SantasBagsProblem, hill_climbing, simulated_annealing
from search import fill_all_bags
import sys
sys.path.append('../common')
from utils import weight3 as weight_fn
from utils import bag_weight, score
from utils import MAX_WEIGHT, AVAILABLE_GIFTS, GIFT_TYPES, N_TYPES, N_BAGS

In [4]:
fixed_weights = {}
for i, g in enumerate(GIFT_TYPES):
    fixed_weights[g] = weight_fn(i, 5000) - 0.15
    
fixed_weights['coal'] = 42
fixed_weights

{'ball': 1.8471894467886858,
 'bike': 19.800921855963754,
 'blocks': 11.587347492790292,
 'book': 1.8620882866801276,
 'coal': 42,
 'doll': 4.8448987856816599,
 'gloves': 1.2071312000834755,
 'horse': 4.8825970336724822,
 'train': 9.9182827557604352}

In [5]:
n_bags_per_state = 1
empty_state = tuple([tuple([0]*N_TYPES)]*n_bags_per_state)

In [6]:
def fix_weight(gift_index):
    return fixed_weights[GIFT_TYPES[gift_index]]

def bag_fix_weight(bag):
    out = 0
    for i, c in enumerate(bag):
        out += fix_weight(i) * c
    return out

def state_fix_score(state):
    score = 0 
    for bag in state:
        score += bag_fix_weight(bag)
    return score

type_cost = {}
for g in fixed_weights:
    type_cost[g] = 1.0/fixed_weights[g]
#     type_cost[g] = (MAX_WEIGHT - fixed_weights[g]) / MAX_WEIGHT 
type_cost

{'ball': 0.54136298891187729,
 'bike': 0.050502699181089607,
 'blocks': 0.086301027963665136,
 'book': 0.53703146470185681,
 'coal': 0.023809523809523808,
 'doll': 0.20640266066142465,
 'gloves': 0.82841036660376932,
 'horse': 0.20480903771160539,
 'train': 0.10082390516838315}

In [11]:
# Penalize states that heavy than goal score 
def h1(state, limit):    
    w = state_fix_score(state)
    if w < limit:
        return (limit - w) / limit
    else:
        return 1.0
    
    
# Difference between fix / 'real' masses
def h2(state):
    w2 = state_fix_score(state)
    w1 = score(state)
    if w2 > 0:
        return abs(w2 - w1)*1.0 / w2
    else:
        return 1.0


# Count number of rejected bags :
def h3(state):
    _, rejected = score(state, return_rejected=True)
    return rejected

In [21]:
alpha = 0.7
goal_score = n_bags_per_state*MAX_WEIGHT*alpha
limit = goal_score + 0.5*(MAX_WEIGHT-goal_score)

initial_state = ((0,0,0,0,0,0,0,0,0,),)

In [23]:
def value(state):
    return h1(state, limit) + h2(state) + h3(state)

In [24]:
p = SantasBagsProblem(initial=initial_state,
                      gift_types=GIFT_TYPES, 
                      available_gifts=AVAILABLE_GIFTS,
                      max_weight=MAX_WEIGHT,    
                      type_cost=type_cost,
                      gift_weight_fn=fix_weight,
                      bag_weight_fn=bag_fix_weight,
                      goal_score=goal_score, 
                      score_fn=score,
                      verbose_level=0)

p.value = value

In [25]:
# hill_climbing(p)
result = simulated_annealing(p)
result

<Node ((1, 0, 0, 0, 0, 1, 3, 2, 3),): 3.9450865403129702>

In [26]:
score(result.state), score(initial_state)

(18.049125507135056, 0.0)

In [32]:
for t in range(sys.maxsize):
    T = exp_schedule(t)
    print(t, T)
    break

0 <function exp_schedule.<locals>.<lambda> at 0x10516bf28>


In [27]:
import math

def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)


def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)


def simulated_annealing(problem, schedule=exp_schedule()):
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current
        neighbors = current.expand(problem)
        if not neighbors:
            return current
        next = random.choice(neighbors)
        delta_e = problem.value(next.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next
